# DistilBERT Model - Sequence Classification

In [ ]:
!pip install transformers -q

     |████████████████████████████████| 127 kB 7.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/HackathonGroup4/checkpoint

config.json  tf_model.h5


In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification, TFTrainingArguments, TFTrainer
from tensorflow.data import Dataset
import numpy as np

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=20,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    weight_decay=0.01,    
    eval_steps=10,
    evaluation_strategy='steps',          
)

with training_args.strategy.scope():
  new_model = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/HackathonGroup4/checkpoint", num_labels=10)

new_trainer = TFTrainer(model=new_model, args=training_args)

Some layers from the model checkpoint at /content/drive/MyDrive/HackathonGroup4/checkpoint were not used when initializing TFDistilBertForSequenceClassification: ['dropout_199']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/HackathonGroup4/checkpoint and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/lo

In [ ]:
#@title test your predictions here! \o/

text = 'call ernest phone' #@param {type:"string"}

mapper = {0: 'orbit', 2: 'enghub', 7: 'techrisk', 8: 'teutr', 6: 'mail', 5: 'kerb', 9: 'time', 3: 'help', 4: 'im', 1: 'call'}
input_encodings = tokenizer([text], truncation=True, padding=True)
input_dataset = Dataset.from_tensor_slices((dict(input_encodings), [0]))
input_predictions = new_trainer.predict(input_dataset)

print(input_predictions)
print("Predicted Command:", mapper[np.argmax(input_predictions.predictions)])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

# Named Entity Recognition

In [ ]:
!pip install boto3

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

print(nltk.__version__)

     |████████████████████████████████| 131 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 7.7 MB 8.8 MB/s 
     |████████████████████████████████| 138 kB 45.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.6 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping ta

In [ ]:
def identify_name_location(command):
  command = "how to "+command
  command = command.title()
  
  ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(command)))
  for node in ne_tree:
    if(type(node) == nltk.tree.Tree and (node.label() == 'PERSON' or node.label() == 'GPE')):
      name = ""
      for child in node:
        name+=child[0] + " "
      return name.rstrip()

identify_name_location('call ernest phone')
print(identify_name_location("chat with binitha"))
print(identify_name_location("how to email siu hong?")) # Failing test case
print(identify_name_location("what is callista's kerberos?"))
print(identify_name_location("what is the time now in london?"))
print(identify_name_location("what is bengaluru's time?"))
print(identify_name_location("what is for lunch in hong kong?")) # Failing test case

print(identify_name_location("skype callista"))
print(identify_name_location("call siu hong"))
print(identify_name_location("kerberos pengfei"))

Binitha
Email Siu Hong
Callista
London
Bengaluru
Lunch
Callista
Siu Hong
Pengfei


In [ ]:
import boto3

def retrieve_kerberos(name, access_key, secret_access_key):

  if(not name):
    return ""

  # Get the service resource.
  client = boto3.client(
      'dynamodb',
      aws_access_key_id=access_key,
      aws_secret_access_key=secret_access_key,
      region_name = 'us-east-2'
    )

  response = client.scan(
        TableName='kerbuser',
        Select='ALL_ATTRIBUTES'
    )
  
  users = response['Items']
  for user in users:
    if(name.lower() in user['name']['S'].lower()):
      return user['kerberos']['S']

  return ""

access_key = ""
secret_access_key = ""
print(retrieve_kerberos("Callista",access_key,secret_access_key))

chcall


In [ ]:
def retrieve_kerberos(name):

  if(not name):
    return ""

  # Get the service resource.
  sts_client = boto3.client('sts')
  sts_client_response = sts_client.assume_role(RoleArn=ROLE_ARN,RoleSessionName="lambdaSession")

  db = boto3.client(
  'dynamodb',
  aws_access_key_id=sts_client_response["Credentials"]["AccessKeyId"],
  aws_secret_access_key=sts_client_response["Credentials"]["SecretAccessKey"],
  aws_session_token=sts_client_response["Credentials"]["SessionToken"])
       

  response = db.scan(
        TableName='kerbuser',
        Select='ALL_ATTRIBUTES'
    )
  
  users = response['Items']
  for user in users:
    if(name.lower() in user['name']['S'].lower()):
      return user['kerberos']['S']
  return ""

In [ ]:
def identify_query(command):
  command = "how to "+command
  command = command.lower()
  ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(command)))
  word_list = []
  for nodes in ne_tree.pos():
    for node in nodes:
      if(type(node) == tuple and (node[1] == 'NN' or node[1] == 'VB')):
          word_list.append(node[0])
    
  return " ".join(word_list)

print(identify_query("How to reinstall my browser?"))
print(identify_query("How to resolve gs app store issues?"))
print(identify_query("How to reboot vscode?"))
print(identify_query("How to add wordart on powerpoint?"))
print(identify_query("How to send zoom invite through outlook"))
print(identify_query("How to split a document in microsoft word"))

reinstall browser
resolve store
reboot vscode
add wordart powerpoint
send zoom invite outlook
split document word
